## Librerias

In [2]:
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
import json
# import plotly.express as px
# import seaborn as sbs

import warnings
warnings.filterwarnings("ignore")

## Parametros

In [3]:
client_bq = bigquery.Client()

In [67]:
cohorte = 4

main_table = f'co-keralty-costomedico.Diabetes_avicena_survival.diabetes_{cohorte}_annos'
final_table = f'Diabetes_avicena_survival.diabetes_final_{cohorte}_annos'

print(f'Se traen los datos desde la tabla: {main_table}')
print(f'Los datos finales para el modelo se guardan en: {final_table}')

Se traen los datos desde la tabla: co-keralty-costomedico.Diabetes_avicena_survival.diabetes_4_annos
Los datos finales para el modelo se guardan en: Diabetes_avicena_survival.diabetes_final_4_annos


In [68]:
query_for_t0_pop = f"""
SELECT numero_identificacion_paciente,
  year,
  month,
  fecha,
  edad,
  peso,
  talla,
  peso/(talla*talla) AS IMC,
  HDL,
  LDL,
  trigliceridos,
  perimetro_abdominal,
  genero_paciente,
  raza_paciente,
  nivel_academico_paciente,
  ant_cardio,
  med_hipertension,
  ant_familiar_dm,
  hace_ejercicio,
  diabetes FROM (SELECT
  numero_identificacion_paciente,
  year,
  month,
  DATE(CONCAT(year,'-',month,'-01')) AS fecha,
  ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY DATE(CONCAT(year,'-',month,'-01')) ASC) AS orden,
  edad,
  peso,
  talla,
  peso/(talla*talla) AS IMC,
  HDL,
  LDL,
  trigliceridos,
  perimetro_abdominal,
  genero_paciente,
  raza_paciente,
  nivel_academico_paciente,
  ant_cardio,
  med_hipertension,
  ant_familiar_dm,
  hace_ejercicio,
  diabetes,
FROM
  `{main_table}`
WHERE
  peso IS NOT NULL
  AND talla IS NOT NULL
  AND genero_paciente IS NOT NULL
  -- AND raza_paciente IS NOT NULL
  -- AND nivel_academico_paciente IS NOT NULL
  AND HDL IS NOT NULL
  AND LDL IS NOT NULL
  AND trigliceridos IS NOT NULL
  -- AND perimetro_abdominal IS NOT NULL
  -- AND hace_ejercicio IS NOT NULL
  AND diabetes = 0)
  WHERE orden = 1
"""

select_pob = f"""
SELECT DISTINCT numero_identificacion_paciente FROM
({query_for_t0_pop})

"""

query_for_t1_pop = """

WITH
  no_diabetes_t1 AS (
  SELECT
    numero_identificacion_paciente,
    year,
    month,
    DATE(CONCAT(year,'-',month,'-01')) AS fecha,
    edad,
    peso,
    talla,
    peso/(talla*talla) AS IMC,
    HDL,
    LDL,
    trigliceridos,
    perimetro_abdominal,
    genero_paciente,
    raza_paciente,
    nivel_academico_paciente,
    ant_cardio,
    med_hipertension,
    ant_familiar_dm,
    hace_ejercicio,
    diabetes,
  FROM (
    SELECT
      *,
      ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY DATE(CONCAT(year,'-',month,'-01')) DESC) AS orden
    FROM
      `{}`
    WHERE
      diabetes = 0
      AND numero_identificacion_paciente IN ({}))
  WHERE
    orden = 1),

si_diabetes_t1 AS (
  SELECT
    numero_identificacion_paciente,
    year,
    month,
    DATE(CONCAT(year,'-',month,'-01')) AS fecha,
    edad,
    peso,
    talla,
    peso/(talla*talla) AS IMC,
    HDL,
    LDL,
    trigliceridos,
    perimetro_abdominal,
    genero_paciente,
    raza_paciente,
    nivel_academico_paciente,
    ant_cardio,
    med_hipertension,
    ant_familiar_dm,
    hace_ejercicio,
    diabetes,
  FROM (
    SELECT
      *,
      ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY DATE(CONCAT(year,'-',month,'-01')) ASC) AS orden
    FROM
      `{}`
    WHERE
      diabetes = 1
      AND numero_identificacion_paciente IN ({}))
  WHERE
    orden = 1)

    
SELECT
  *
FROM
  no_diabetes_t1
WHERE
  no_diabetes_t1.numero_identificacion_paciente NOT IN (
  SELECT
    DISTINCT numero_identificacion_paciente
  FROM
    si_diabetes_t1)
UNION ALL (
  SELECT
    *
  FROM
    si_diabetes_t1)

"""

In [69]:
print(query_for_t0_pop)


SELECT numero_identificacion_paciente,
  year,
  month,
  fecha,
  edad,
  peso,
  talla,
  peso/(talla*talla) AS IMC,
  HDL,
  LDL,
  trigliceridos,
  perimetro_abdominal,
  genero_paciente,
  raza_paciente,
  nivel_academico_paciente,
  ant_cardio,
  med_hipertension,
  ant_familiar_dm,
  hace_ejercicio,
  diabetes FROM (SELECT
  numero_identificacion_paciente,
  year,
  month,
  DATE(CONCAT(year,'-',month,'-01')) AS fecha,
  ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY DATE(CONCAT(year,'-',month,'-01')) ASC) AS orden,
  edad,
  peso,
  talla,
  peso/(talla*talla) AS IMC,
  HDL,
  LDL,
  trigliceridos,
  perimetro_abdominal,
  genero_paciente,
  raza_paciente,
  nivel_academico_paciente,
  ant_cardio,
  med_hipertension,
  ant_familiar_dm,
  hace_ejercicio,
  diabetes,
FROM
  `co-keralty-costomedico.Diabetes_avicena_survival.diabetes_4_annos`
WHERE
  peso IS NOT NULL
  AND talla IS NOT NULL
  AND genero_paciente IS NOT NULL
  -- AND raza_paciente IS NOT NU

## Funciones

In [70]:
def load_data(client, query, local = False, save = True, file_name = 'data'):

    if local:
        data = pd.read_csv(f'data/{file_name}.csv', sep = '|')
    else:
        data = client.query(query).result().to_dataframe()

    if save:
        data.to_csv(f'data/{file_name}.csv', sep = '|')

    return data

def etiquetar_diabetes(t0,t1):
    if t1 > t0:
        return t1
    else:
        return t0

## Proceso

In [71]:
fechas_minimas = client_bq.query(f'SELECT DISTINCT year, month FROM `{main_table}` ORDER BY 1, 2 LIMIT 1').result().to_dataframe()
min_year = fechas_minimas['year'].values[0]
min_month = fechas_minimas['month'].values[0]

print(f'El tiempo inicial del estudio es {min_year}-{min_month}')

El tiempo inicial del estudio es 2021-2


In [58]:
# Carga de afiliados con datos en t0
data_t0 = load_data(client=client_bq, query=query_for_t0_pop, file_name='data_t0')
print(f'Se cuentan con {data_t0.shape[0]} afiliados con datos en el momento t0')
display(data_t0.head())

# Carga de afiliados que cumplen el evento (Diabetes)
# afiliados_con_t0 = tuple(data_t0.numero_identificacion_paciente.unique())
data_t1 = load_data(client=client_bq, query=query_for_t1_pop.format(main_table,select_pob,main_table,select_pob), file_name='data_t1')
print(f'Se cuentan con {data_t1.shape[0]} afiliados que cumplen el evento (Diabetes) momento t1')
display(data_t1.head())

Se cuentan con 834246 afiliados con datos en el momento t0


,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
0,1003390652,2022,2,2022-02-01,26,105.233333333,1.600000000,41.106770833,71.20,106.14,117.80,111.000000000,Femenino,Mestizo,Bachillerato técnico,0,0,0,None,0
1,1006578626,2022,2,2022-02-01,20,68.700000000,1.770000000,21.928564589,56.25,143.00,100.05,None,Masculino,Otros,Profesional,1,0,0,None,0
2,1007218577,2022,2,2022-02-01,28,74.300000000,1.780000000,23.450321929,45.10,96.64,64.68,None,Masculino,Otros,Ninguno,0,0,0,None,0
3,1010021506,2022,2,2022-02-01,36,71.000000000,1.600000000,27.734375000,60.00,140.00,117.00,None,Masculino,Otros,Tecnológica,1,1,0,None,0
4,10241950,2022,2,2022-02-01,67,85.800000000,1.720000000,29.002163332,41.00,82.00,203.00,105.000000000,Masculino,Otros,Básica secundaria,1,1,0,None,0


Se cuentan con 834246 afiliados que cumplen el evento (Diabetes) momento t1


,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
0,1015460210,2022,12,2022-12-01,27,52.000000000,1.590000000,20.568806614,NaN,NaN,NaN,None,Femenino,Otros,Profesional,0,0,0,None,1
1,1047490387,2022,12,2022-12-01,26,66.000000000,1.680000000,23.384353741,NaN,NaN,NaN,None,Femenino,Otros,Profesional,0,0,0,None,1
2,11310920,2022,12,2022-12-01,58,83.800000000,1.690000000,29.340709359,NaN,NaN,NaN,99.000000000,Masculino,Otros,Básica secundaria,0,0,0,None,1
3,11807801,2022,12,2022-12-01,44,94.000000000,1.740000000,31.047694544,27.0,112.0,130.0,None,Masculino,Afrocolombiano,Ninguno,1,0,0,20 minutos\n,1
4,21768447,2022,12,2022-12-01,70,47.800000000,1.440000000,23.051697531,NaN,NaN,NaN,None,Femenino,Otros,Básica secundaria,1,1,0,None,1


In [59]:
len(data_t0.numero_identificacion_paciente.unique())

834246

In [60]:
afiliados_with_no_t1 = list(data_t0[data_t0.fecha >= data_t1.fecha].numero_identificacion_paciente.unique())

data_t1_clean = data_t1[~data_t1.numero_identificacion_paciente.isin(afiliados_with_no_t1)]
data_t0_clean = data_t0[~data_t0.numero_identificacion_paciente.isin(afiliados_with_no_t1)]
data_t1_clean_to_merge = data_t1_clean[['numero_identificacion_paciente','diabetes']].rename(columns = {'diabetes':'diabetes_t1'})
full_data = data_t0_clean.merge(data_t1_clean_to_merge, on = 'numero_identificacion_paciente', how = 'left') #pd.concat([data_t0_clean,data_t1_clean])
full_data.diabetes = full_data.apply(lambda x: etiquetar_diabetes(x.diabetes, x.diabetes_t1), axis = 1)
full_data = full_data.drop(columns = 'diabetes_t1')

full_data

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
0,1003390652,2022,2,2022-02-01,26,105.233333333,1.600000000,41.106770833,71.20,106.14,117.80,111.000000000,Femenino,Mestizo,Bachillerato técnico,0,0,0,None,0
1,1006578626,2022,2,2022-02-01,20,68.700000000,1.770000000,21.928564589,56.25,143.00,100.05,None,Masculino,Otros,Profesional,1,0,0,None,0
2,1007218577,2022,2,2022-02-01,28,74.300000000,1.780000000,23.450321929,45.10,96.64,64.68,None,Masculino,Otros,Ninguno,0,0,0,None,0
3,1010021506,2022,2,2022-02-01,36,71.000000000,1.600000000,27.734375000,60.00,140.00,117.00,None,Masculino,Otros,Tecnológica,1,1,0,None,0
4,10241950,2022,2,2022-02-01,67,85.800000000,1.720000000,29.002163332,41.00,82.00,203.00,105.000000000,Masculino,Otros,Básica secundaria,1,1,0,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804383,92505045,2025,1,2025-01-01,60,82.250000000,1.650000000,30.211202938,45.10,130.00,236.00,103.500000000,Masculino,Otros,Bachillerato técnico,0,1,0,Nunca\n,0
804384,93397252,2025,1,2025-01-01,50,95.200000000,1.750000000,31.085714286,28.90,125.39,137.77,None,Masculino,Otros,Ninguno,1,1,0,20 minutos\n,0
804385,94064517,2025,1,2025-01-01,42,113.500000000,1.850000000,33.162892622,48.00,187.00,78.00,None,Masculino,Otros,Ninguno,0,0,0,None,0
804386,94455458,2025,1,2025-01-01,49,80.050000000,1.750000000,26.138775510,51.00,186.00,268.00,None,Masculino,Otros,Ninguno,0,0,0,40 minutos\n,0


In [61]:
full_data[full_data.numero_identificacion_paciente == '1121827561'] #72020868

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
330335,1121827561,2022,12,2022-12-01,36,127.000000000,1.960000000,33.059142024,38.0,76.4,158.0,119.000000000,Masculino,Otros,Bachillerato técnico,1,1,0,Nunca\n,0


In [62]:
full_data[full_data.numero_identificacion_paciente == '79569003'] #72020868

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
497153,79569003,2023,9,2023-09-01,52,93.500000000,1.870000000,26.737967914,45.0,106.0,124.0,97.000000000,Masculino,Otros,Básica secundaria,1,0,0,Nunca\n,1


In [63]:
fechas = data_t1_clean[['numero_identificacion_paciente','year','month','fecha']].rename(columns = {'year':'year_t1','month':'month_t1'})

fechas['year_event'] = fechas.fecha.apply(lambda x: x.year)
fechas['month_event'] = fechas.fecha.apply(lambda x: x.month)
fechas.head(1)

,numero_identificacion_paciente,year_t1,month_t1,fecha,year_event,month_event
0,1015460210,2022,12,2022-12-01,2022,12


In [64]:
times_to_event = data_t0_clean.sort_values(['numero_identificacion_paciente','year','month'])[['numero_identificacion_paciente','year','month','diabetes']]
times_to_event = times_to_event.merge(fechas, on = 'numero_identificacion_paciente', how = 'left')
times_to_event['time_to_event'] = abs((times_to_event.year - times_to_event.year_event)*12 + (times_to_event.month - times_to_event.month_event))
times_to_event = times_to_event[times_to_event.time_to_event != 0][['numero_identificacion_paciente','time_to_event']]

times_to_event

,numero_identificacion_paciente,time_to_event
0,1000000426,8
1,1000000915,5
2,1000000947,11
3,1000001870,32
5,1000002872,28
...,...,...
804382,D20193951,24
804383,D2023160,9
804385,NRRC6PK77,1
804386,NU72CKF61,21


In [65]:
full_data_with_time = full_data.merge(times_to_event, on = 'numero_identificacion_paciente', how = 'inner')
full_data_with_time[['peso','talla','perimetro_abdominal']] = full_data_with_time[['peso','talla','perimetro_abdominal']].astype(float)
full_data_with_time.hace_ejercicio = full_data_with_time.hace_ejercicio.apply(lambda x: x if x is None else x.replace('\n',''))

full_data_with_time

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,...,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes,time_to_event
0,1003390652,2022,2,2022-02-01,26,105.233333,1.60,41.106770833,71.20,106.14,...,111.0,Femenino,Mestizo,Bachillerato técnico,0,0,0,None,0,34
1,1006578626,2022,2,2022-02-01,20,68.700000,1.77,21.928564589,56.25,143.00,...,NaN,Masculino,Otros,Profesional,1,0,0,None,0,36
2,1007218577,2022,2,2022-02-01,28,74.300000,1.78,23.450321929,45.10,96.64,...,NaN,Masculino,Otros,Ninguno,0,0,0,None,0,33
3,1010021506,2022,2,2022-02-01,36,71.000000,1.60,27.734375000,60.00,140.00,...,NaN,Masculino,Otros,Tecnológica,1,1,0,None,0,35
4,10241950,2022,2,2022-02-01,67,85.800000,1.72,29.002163332,41.00,82.00,...,105.0,Masculino,Otros,Básica secundaria,1,1,0,None,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734998,80503895,2025,1,2025-01-01,52,97.000000,1.69,33.962396275,30.00,116.00,...,105.0,Masculino,Otros,Básica secundaria,0,0,0,None,1,21
734999,8298242,2025,1,2025-01-01,76,58.000000,1.60,22.656250000,47.90,65.62,...,72.0,Masculino,Otros,Profesional,0,0,0,20 minutos,1,35
735000,8670023,2025,1,2025-01-01,70,66.000000,1.70,22.837370242,42.00,58.00,...,NaN,Masculino,Otros,Profesional,0,1,0,None,1,35
735001,9068399,2025,1,2025-01-01,82,90.900000,1.77,29.014650962,39.00,154.00,...,NaN,Masculino,Otros,Profesional,1,1,0,None,0,1


In [66]:
pandas_gbq.to_gbq(full_data_with_time, final_table, project_id='co-keralty-costomedico',if_exists = 'replace')

100%|██████████| 1/1 [00:00<00:00, 7943.76it/s]


In [38]:
full_data_with_time[full_data_with_time.numero_identificacion_paciente == '1121827561'] #72020868

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,...,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes,time_to_event
150662,1121827561,2022,12,2022-12-01,36,127.0,1.96,33.059142024,38.0,76.4,...,119.0,Masculino,Otros,Bachillerato técnico,1,1,0,Nunca,0,16


In [61]:
full_data[full_data.numero_identificacion_paciente == '1121827561'] #72020868

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
